## CA 4 - Part 2, LLMs Spring 2025

- **Name:**
- **Student ID:**

---
#### Your submission should be named using the following format: `CA4_LASTNAME_STUDENTID.ipynb`.

---

TA Email: miladmohammadi@ut.ac.ir

##### *How to do this problem set:*

- Some questions require writing Python code and computing results, and the rest of them have written answers. For coding problems, you will have to fill out all code blocks that say `YOUR CODE HERE`.

- For text-based answers, you should replace the text that says ```Your Answer Here``` with your actual answer.

- There is no penalty for using AI assistance on this homework as long as you fully disclose it in the final cell of this notebook (this includes storing any prompts that you feed to large language models). That said, anyone caught using AI assistance without proper disclosure will receive a zero on the assignment (we have several automatic tools to detect such cases). We're literally allowing you to use it with no limitations, so there is no reason to lie!

---

##### *Academic honesty*

- We will audit the Colab notebooks from a set number of students, chosen at random. The audits will check that the code you wrote actually generates the answers in your notebook. If you turn in correct answers on your notebook without code that actually generates those answers, we will consider this a serious case of cheating.

- We will also run automatic checks of Colab notebooks for plagiarism. Copying code from others is also considered a serious case of cheating.

---

## Text2SQL

In this section, you will progressively build and evaluate multiple Text-to-SQL pipelines. You’ll start with a simple prompting-based baseline, then design a graph-based routing system using chain-of-thought and schema reasoning, and finally construct a ReAct agent that interacts with the schema via tools. Each stage demonstrates a different strategy for generating SQL from natural language using LLMs.

### Initializations

This section prepares the environment and initializes the LLM model (Gemini) to be used in later parts of the notebook.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### Load API Key (2 Points)

**Task:** Load the Gemini API key stored in the `.env` file and set it as an environment variable so it can be used to authenticate API requests later.

* Use `dotenv` to load the file.
* Extract the API key with `os.getenv`.

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
  raise ValueError("GOOGLE_API_KEY not found in .env file or environment variable")

#### Create ChatModel (3 Points)

**Task:** Create an instance of the Gemini LLM using LangChain. You should configure the model with proper parameters for our task.

Note: You may use any model that supports Structured Output and Tool Use. We recommend using gemini-2.5-flash-preview-05-20 from Google AI Studio, as it offers a generous free tier.

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_param = {
    "model": "gemini-2.5-flash-preview-05-20",
    "temperature": 0.01,
    "max_output_tokens": 512,

}
# Create an instance of the Gemini LLM
llm = ChatGoogleGenerativeAI(model_kwargs=gemini_param, google_api_key=GOOGLE_API_KEY)

C:\Users\sazgar\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3553: UserWarning: Parameters {'model', 'max_output_tokens', 'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
llm.invoke("just say Meow!")

AIMessage(content='Meow!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, id='run--8e0cb2aa-081a-4027-ae87-10eb942529db-0', usage_metadata={'input_tokens': 6, 'output_tokens': 33, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 30}})

### Baseline

In this section, you'll build a simple baseline pipeline that directly converts a question and schema into a SQL query using a single prompt.

#### Baseline Function (5 Points)


**Task:** Implement a function that sends a system message defining the task, and a user message containing the input question and schema. The LLM should return the SQL query formatted as: "```sql\n[query]```"

In [5]:
from langchain_core.messages import SystemMessage, HumanMessage

def run_baseline(question: str, schema: str):
    sys_msg = """
    ### TASK DESCRIPTION
    You are an expert systems that receives a user's question and a database schema. \
    Generate a SQL query that answers the question.

    ### FORMAT
    formatted as a markdown SQL code block:\n
    ```sql
    [query]
    ```
    """
    user_msg = f"""
    Question: {question}\n
    Schema:\n{schema}\n
    Only generate the SQL query, nothing else.
    """

    messages = [
        SystemMessage(content=sys_msg),
        HumanMessage(content=user_msg)
    ]

    response = llm.invoke(messages)
    sql_query = response.content

    return sql_query

#### Run and Evaluate (Estimated Run Time 5-10min)

Run your baseline function over the dataset provided.

In [5]:
from method_run import run_method
import re

def function_template(item):
    result = run_baseline(item['question'], item['schema'])
    # First try to extract query from markdown SQL block
    match = re.search(r'```sql\n(.*?)```', result, re.DOTALL)
    if match:
        query = match.group(1).strip()
    else:
        # If no markdown block found, try to extract just SQL query
        query = result.strip()
        # Remove any ```sql or ``` if present without proper formatting
        query = re.sub(r'```sql|```', '', query).strip()

    print(f"Question: {item['question']}")
    print(f"Schema: {item['schema']}")
    print(f"Generated SQL: {query}\n")

    return {**item, 'sql': query}

run_method(function_template, SLEEP_TIME=10)

#Run on mode=nano if you want to test it on a smaller dataset
#run_method(function_template, SLEEP_TIME=10, mode="nano")

  0%|          | 0/18 [00:00<?, ?it/s]

Question: Find the percentage of atoms with single bond. (Evidence: single bond refers to bond_type = '-'; percentage = DIVIDE(SUM(bond_type = '-'), COUNT(bond_id)) as percentage)
Schema: atom (atom_id, molecule_id, element)
bond (bond_id, molecule_id, bond_type)
connected (atom_id, atom_id2, bond_id)
molecule (molecule_id, label)

Generated SQL: SELECT
  CAST(SUM(CASE WHEN bond_type = '-' THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(bond_id)
FROM bond;



  6%|▌         | 1/18 [00:11<03:09, 11.15s/it]

Question: Indicate which atoms are connected in non-carcinogenic type molecules. (Evidence: label = '-' means molecules are non-carcinogenic)
Schema: atom (atom_id, molecule_id, element)
bond (bond_id, molecule_id, bond_type)
connected (atom_id, atom_id2, bond_id)
molecule (molecule_id, label)

Generated SQL: SELECT
  T1.atom_id,
  T1.atom_id2
FROM connected AS T1
INNER JOIN bond AS T2
  ON T1.bond_id = T2.bond_id
INNER JOIN molecule AS T3
  ON T2.molecule_id = T3.molecule_id
WHERE
  T3.label = '-';



 11%|█         | 2/18 [00:23<03:06, 11.64s/it]

...

Question: Calculate the total average cost that Elijah Allen spent in the events on September and October. (Evidence: events in September and October refers to month(expense_date) = 9 AND MONTH(expense_date) = 10)
Schema: event (event_id, event_name, event_date, type, notes, location, status)
major (major_id, major_name, department, college)
zip_code (zip_code, type, city, county, state, short_state)
attendance (link_to_event, link_to_member)
budget (budget_id, category, spent, remaining, amount, event_status, link_to_event)
expense (expense_id, expense_description, expense_date, cost, approved, link_to_member, link_to_budget)
income (income_id, date_received, amount, source, notes, link_to_member)
member (member_id, first_name, last_name, email, position, t_shirt_size, phone, zip, link_to_major)

Generated SQL: SELECT
  AVG(T1.cost)
FROM expense AS T1
INNER JOIN member AS T2
  ON T1.link_to_member = T2.member_id
WHERE
  T2.first_name = 'Elijah' AND T2.last_name = 'Allen' AND (
    STR

 94%|█████████▍| 17/18 [03:23<00:11, 11.85s/it]

Question: Find the name and date of events with expenses for pizza that were more than fifty dollars but less than a hundred dollars. (Evidence: name of event refers to event_name; date of event refers to event_date; expenses for pizza refers to expense_description = 'Pizza' where cost > 50 and cost < 100)
Schema: event (event_id, event_name, event_date, type, notes, location, status)
major (major_id, major_name, department, college)
zip_code (zip_code, type, city, county, state, short_state)
attendance (link_to_event, link_to_member)
budget (budget_id, category, spent, remaining, amount, event_status, link_to_event)
expense (expense_id, expense_description, expense_date, cost, approved, link_to_member, link_to_budget)
income (income_id, date_received, amount, source, notes, link_to_member)
member (member_id, first_name, last_name, email, position, t_shirt_size, phone, zip, link_to_major)

Generated SQL: SELECT
  T1.event_name,
  T1.event_date
FROM event AS T1
INNER JOIN budget AS T2
 

100%|██████████| 18/18 [03:35<00:00, 11.95s/it]


Starting to compare without knowledge for ex
Process finished successfully
start calculate
                     simple               moderate             challenging          total               
count                6                    6                    6                    18                  
======================================    ACCURACY    =====================================
accuracy             66.67                83.33                83.33                77.78               
Finished evaluation



### Chain/Router

Here, you will build a more advanced system that routes the query through different paths based on question difficulty. Easier questions go straight to query generation; harder ones go through schema path extraction first.

#### Define State (5 Points)

**Task:** Define a `RouterGraphState` using `MessagesState` and `pydantic` that contains:
* The input question and schema
* The predicted difficulty level
* The extracted schema path
* The final query

In [6]:
from langgraph.graph import MessagesState
from typing import Literal, Optional
from pydantic import Field

class RouterGraphState(MessagesState):
    question: str = Field(
        ..., 
        title="User Question", 
        description="The natural language question that needs to be converted into an SQL query."
    )
    
    schema: str = Field(
        ..., 
        title="Database Schema", 
        description="The full database schema context provided to the LLM."
    )
    
    question_difficulty: Literal["simple", "moderate", "challanging"] = Field(
        ..., 
        title="Question Difficulty",
        description="Classification of the question's difficulty used to determine the query generation path."
    )
    
    schema_path: Optional[str] = Field(
        default=None,
        title="Extracted Schema Path",
        description="The relevant subset of the schema used for query generation when the question is classified as hard."
    )
    
    query: Optional[str] = Field(
        default=None,
        title="Generated SQL Query",
        description="The final SQL query generated based on the question and schema."
    )

#### Node: Analyser (5 Points)

**Task:** Build a node that:
* Accepts a question and schema
* Analyzes the difficulty (simple/moderate/challanging)
* Uses the LLM’s structured output feature to return the difficulty

**Steps**:

1. Define a Pydantic class to hold the expected structured output.
2. Use structure output mode of LLM to bind it to the model.

In [7]:
from pydantic import BaseModel

class QuestionDifficaultyAnalysis(BaseModel):
    question_difficulty: Literal["simple", "moderate", "challanging"] = Field(
        ..., 
        title="Question Difficulty Level",
        description=(
            "The estimated difficulty of the SQL question. "
            "'simple' means a direct query on one table with no joins or aggregation. "
            "'moderate' may require joins or basic filters. "
            "'challenging' involves complex logic, multiple joins, nested queries, or interpretation of intent."
        )
    )

    reasoning: str = Field(
        ..., 
        title="Reasoning",
        description="Brief explanation of why this difficulty was assigned."
    )

def analyser_node(state: RouterGraphState):
    sys_msg = SystemMessage(
        content="""
        ### TASK DESCRIPTION
            You are an expert system that analyzes a user's natural language question and a database schema.
            Your sole task is to classify the question's difficulty level as one of: "simple", "moderate", or "challanging".

            Judge the difficulty based on the following criteria:
            - **Simple:** The query requires only one or two tables, straightforward filters (WHERE clauses), and basic aggregations (COUNT, SUM, AVG) without complex joins or nested queries.
            - **Moderate:** The query requires multiple tables, involves JOIN operations, might include more complex filtering, basic grouping (GROUP BY), or simple ORDER BY clauses.
            - **Challenging:** The query involves multiple complex JOINs, subqueries, complex aggregations, conditional logic (CASE statements), or advanced SQL functions/patterns.

            Your response must be a JSON object containing two keys:
            1. "question_difficulty" with the classified value ("simple" | "moderate" | "challanging").
            2. "reasoning" with a brief explanation of *why* this difficulty was assigned, referencing the criteria.

        ### FORMAT
            ```json
            {"question_difficulty": "simple" | "moderate" | "challanging", "reasoning": "Explanation here..."}
            ```
        """
    )

    user_msg = HumanMessage(
        content=f"""
        ### Question
        {state.question}\n
        ### Schema
        {state.schema}\n
        Only generate the JSON object, nothing else.
        """
    )
    messages = [sys_msg, user_msg]
    llm_struct = llm.with_structured_output(QuestionDifficaultyAnalysis)
    response = llm_struct.invoke(messages)
    state.question_difficulty = response.question_difficulty
    return state

#### Conditional Edge (2 Points)

**Task:** Implement a branching function that decides whether to proceed to direct query generation or schema path extraction based on the difficulty label returned by the analyser.

* If the difficulty is “easy”, go directly to query generation.
* Otherwise, extract the schema path first.

In [8]:
def is_schema_extraction_needed(state: RouterGraphState) -> Literal["schema_path_extractor", "query_generator"]:
    """
    Routing decision function for LangGraph.

    Determines the next processing step based on the predicted difficulty level of the question.

    Returns:
        - "query_generator": for simple questions that require no schema reduction.
        - "schema_path_extractor": for moderate or challenging questions that benefit from targeted schema focus.

    Raises:
        ValueError: If the difficulty label is missing or invalid.
    """

    difficulty = state.question_difficulty 
    
    if state.schema_path is not None:
        return "query_generator" 

    if not isinstance(difficulty, str):
        raise TypeError("Difficulty must be a string literal ('simple', 'moderate', or 'challenging').") 

    difficulty_normalized = difficulty.strip().lower() 

    if difficulty_normalized == "simple":
        return "query_generator"

    elif difficulty_normalized in {"moderate", "challenging"}:
        return "schema_path_extractor"

    else:
        raise ValueError(
            f"Unrecognized difficulty label: '{difficulty}'. "
            "Expected one of: 'simple', 'moderate', 'challenging'."
        )

#### Node: Schema Extractor (3 Points)

**Task:** Implement a node that takes the question and schema and extracts a join path or sequence of relevant tables from the schema based on the question.

* Use a simple prompt for this.
* Store the result in the `schema_path` field of the state.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

schema_path_prompt = ChatPromptTemplate.from_messages([
    (
            "system",
            "You are a SQL database schema extraction specialist. Your ONLY task is to identify and return the exact table schemas needed to answer the user's question.\n\n"
            
            "EXTRACTION RULES:\n"
            "1. Analyze the question to identify which tables contain the data needed\n"
            "2. Include ALL columns from identified tables (do not filter columns)\n"
            "3. Include related tables connected by foreign keys\n"
            "4. Return and copy schemas in their EXACT original format\n"
            "5. If uncertain about relevance, include the table (better to over-include than miss required data)\n"
            "6. Never made up schemas which not exists! Pay attention fully to question details.\n\n"
            "OUTPUT FORMAT:\n"
            "- TRY TO Return ONLY the relevant Schema based on information provided in question.\n"
            "- DO NOT wrap output in markdown code blocks (no ```sql or ``` tags)\n"
            "- DO NOT add any formatting markers or backticks\n"
            "- Return raw Schemas only\n"
            "- Preserve exact formatting, syntax, and structure\n"
            "- No explanations, comments, or additional text\n\n"
            
            "VALIDATION CHECKLIST:\n"
            "- Does the question mention specific table/column names? Include those tables\n"
            "- Are there JOIN operations implied? Include all related tables\n"
            "- Are there aggregations or filters mentioned? Include tables with those columns\n"
            "- Are there foreign key relationships? Include both parent and child tables\n\n"
            
            "ALWAYS return at least one table schema. If the question is unclear, include the most likely relevant tables rather than returning nothing."),
    ("human",
     "### Question:\n{question}\n\n### Schema:\n{schema}") 
])


schema_path_extractor_chain: Runnable = schema_path_prompt | llm


def schema_path_extractor_node(state: RouterGraphState) -> RouterGraphState:
    """
    Extracts the subset of the schema relevant to the user's question.
    This is used for complex questions that benefit from schema narrowing.
    Updates: state.schema_path
    """
    # Prepare and run prompt    
    result = schema_path_extractor_chain.invoke({
        "question": state.question.strip(),
        "schema": state.schema.strip()
    })
    
    # Sanitize and assign result
    extracted = result.content.strip()
    if not extracted:
        print("Warning: Schema path extraction returned empty. Using full schema as fallback.")
        state.schema_path = state.schema
    else:
        state.schema_path = extracted
    
    return state

#### Node: Generator (5 Points)

**Task:** Generate the SQL query based on the question and schema.

* If a schema path is available, include it in the prompt.
* Save the output query in the `query` field of the state.


In [12]:
query_generation_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     You are an **expert SQL query generator**. Your **ONLY** task is to generate the most accurate SQL query to answer the user's question, \
     using **STRICTLY** the provided schema and, if given, the relevant schema path.

     ### APPLY these rules precisely:
     - Use ONLY tables and columns present in the provided schema or schema path.
     - Do NOT invent, rename, or hallucinate any schema elements.
     - Use JOINs ONLY as needed, based on explicit foreign key relationships or clear requirements from the schema/path.
     - If the question requires aggregation, grouping, subqueries, or ordering, reflect these precisely as per SQL syntax.
     - For simple questions (as inferred from language or difficulty level), \
          use concise SQL and avoid unneeded complexity. For moderate/challenging questions, carefully write correct, \
               efficient, and clear multi-table, subquery, or CASE logic as per requirements.
     - Do NOT output any prose, explanation, comments, or formatting markers—only the SQL query in the format below.

     Output format:
     ```sql
     [YOUR_SQL_QUERY_HERE]
     ```
     """),
    ("human",
     """
     #### Question
     {question}

     #### Schema
     {schema}

     {%- if schema_path and schema_path.strip() %}
     #### Contextual Schema Information
     {{ schema_path }}
     {%- endif %}

     Only return the final SQL query in the specified format.
     """)
])

query_generation_chain: Runnable = query_generation_prompt | llm

def query_generator_node(state: RouterGraphState):
     """
    Generates the final SQL query based on the question, full schema,
    and optionally, the extracted schema path for complex queries.
    Updates: state.query
    """
     # Prepare and run prompt
     inputs = {
          "question": state.question.strip(),
          "schema": state.schema.strip(),
          "schema_path": state.schema_path.strip() if state.schema_path else ""
     }
     result = query_generation_chain.invoke(inputs)
     
     # Sanitize and assign result
     state.query = result.content.strip()
     return state

#### Build Graph (5 Points)

**Task:** Assemble the full routing graph using the nodes and edges you created.

In [ ]:
from langgraph.graph import StateGraph, START

router_graph_builder = StateGraph(RouterGraphState)

#YOUR CODE HERE

router_graph = router_graph_builder.compile()

#### Run and Evaluate (Estimated Run Time 10-15min)

**Task:** Run your compiled routing graph on a dataset. For each question:

* Instantiate the `RouterGraphState` with the question and schema.
* Run the graph to completion.
* Extract and clean the query from the result.

Use the `run_method` function to handle iteration and timing.

In [11]:

from method_run import run_method
def run_router_graph(item):
    response = router_graph.invoke(
        RouterGraphState(
            question=item['question'],
            schema=item['schema'],
            schema_path=None,
            question_difficulty=None,
            query=None
        )
    )
    result = response["query"]
    # First try to extract query from markdown SQL block
    match = re.search(r'```sql\n(.*?)```', result, re.DOTALL)
    if match:
        query = match.group(1).strip()
    else:
        # If no markdown block found, try to extract just SQL query
        query = result.strip()
        # Remove any ```sql or ``` if present without proper formatting
        query = re.sub(r'```sql|```', '', query).strip()
    print(f"Question: {item['question']}")
    print(f"Schema: {item['schema']}")
    print(f"Question Difficulty: {response['question_difficulty']}")
    if response["schema_path"]:
        print(f"Schema Path: {response['schema_path']}")
    print(f"Generated SQL: {query}\n")
    return {**item, 'sql': query}


run_method(run_router_graph, SLEEP_TIME=30)

#Run on mode=nano if you want to test it on a smaller dataset
#run_method(run_router_graph, SLEEP_TIME=10, mode="nano")

  0%|          | 0/18 [00:00<?, ?it/s]


NameError: name 'router_graph' is not defined

### Agent (ReAct)

Now you will implement a full ReAct agent that incrementally solves the Text-to-SQL task using tools. The agent can explore tables and columns before finalizing the query.

**You are not allowed to use 'Prebuilt Agent' of LangGraph. You have to build your own graph.**

#### Define Tools

**Task:** Define three tools for the agent to interact with the schema:
1. `get_samples_from_table`: Returns the first few rows of a table.
2. `get_column_description`: Provides a human-readable description of a specific column.
3. `execute`: Executes a SQL query.

In [10]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import ToolNode

from db_manager import DBManager
db_manager = DBManager()

@tool
def get_samples_from_table(table_name: str, config: RunnableConfig):
  """Gets the first few rows (samples) from a specified table.

  Args:
    table_name: The name of the table from which to fetch samples.

  Returns:
    The first few rows from the specified table.
  """
  db_name = config["configurable"].get("database_name")
  result = db_manager.get_table_head(table_name, db_name=db_name)
  return result

@tool
def get_column_description(table_name: str, column_name: str, config: RunnableConfig):
  """Provides a description for a specific column within a given table.

  Args:
    table_name: The name of the table containing the column.
    column_name: The name of the column for which to get the description.

  Returns:
    A string containing the description of the specified column.
  """
  db_name = config["configurable"].get("database_name")
  result = db_manager.get_column_description(db_name, table_name, column_name)
  return result

@tool
def execute(query: str, config: RunnableConfig):
  """Executes a given SQL query against the database.

  Args:
    query: The SQL query string to be executed.

  Returns:
    The result of the executed query. This could be a set of rows,
    a confirmation message, or an error.
  """
  db_name = config["configurable"].get("database_name")
  result = db_manager.query(query, db_name)
  return result

#### Extra Tool (5+5 Bonus Points):

**Task**: Create and integrate a new custom tool into the ReAct agent. To receive credit for this part, your tool must be meaningfully different from the existing three tools and provide practical value in helping the agent generate more accurate or efficient SQL queries.

In [ ]:
#YOUR CODE HERE

#### Create Tool Node

In [ ]:
tools = [get_samples_from_table, get_column_description, execute]
tools_node = ToolNode(tools=tools)

#### ReAct Agent Prompt (5 Points)

**Task:** Set up the agent node with planning, tool use, and final SQL generation prompts. For writing efficient prompt you can read this link.
https://cookbook.openai.com/examples/gpt4-1_prompting_guide

In [ ]:
REACT_SYS_PROMPT = """
#YOUR PROMPT HERE
"""


#### Agent Node (5 Points)

**Task:** Set up the agent node with models that have binded with tools.

In [ ]:
import time

def agent_node(state: MessagesState) -> MessagesState:
    #For rate-limiting purposes, we will sleep for 10 seconds before invoking the LLM
    time.sleep(10)
    #YOUR CODE HERE
    return state

#### Build Graph (5 Points)

**Task:** Assemble the ReAct agent graph, connecting the agent node and tool node.

In [ ]:
from langgraph.prebuilt import tools_condition
from typing_extensions import TypedDict

class ConfigSchema(TypedDict):
    database_name: str

react_builder = StateGraph(MessagesState, config_schema=ConfigSchema)

#YOUR CODE HERE

react_graph = react_builder.compile()


#### Run and Evaluate (Estimated Run Time 20min)

**Task:** Execute the ReAct agent pipeline on the dataset and collect SQL outputs.

In [ ]:
from method_run import run_method
import re
def run_react_agent_with_config(item):
    question = item['question']
    schema = item['schema']
    user_prompt = f"Question: {question}\nSchema: {schema}"
    input_msg = HumanMessage(content=user_prompt)
    input_config = {"configurable": {"database_name": item['db_id']}}
    response = react_graph.invoke(MessagesState(messages=[input_msg]), config=input_config)

    for msg in response["messages"]:
        msg.pretty_print()

    # If last AI Message is a list of messages, we need to extract the last one
    last_msg = response["messages"][-1].content
    if isinstance(last_msg, list):
        last_msg = last_msg[-1]

    # First try to extract query from markdown SQL block
    match = re.search(r'```sql\n(.*?)```', last_msg, re.DOTALL)
    if match:
        query = match.group(1).strip()
    else:
        # If no markdown block found, try to extract just SQL query
        query = last_msg.strip()
        # Remove any ```sql or ``` if present without proper formatting
        query = re.sub(r'```sql|```', '', query).strip()

    return {**item, 'sql': query}

#Run agent on mode=nano, it's not needed to run on full dataset
run_method(run_react_agent_with_config, SLEEP_TIME=20, mode="nano")